In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1616504514575_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, r.review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2020-01-01' \
limit 100000 \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UGC_ID           100000 non-null  int64  
 1   REVIEW_HEADLINE  99999 non-null   object 
 2   REVIEW_COMMENTS  99484 non-null   object 
 3   LANG             100000 non-null  object 
 4   REVIEW_LENGTH    99484 non-null   float64
 5   FL               100000 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 4.6+ MB

In [6]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
#text = 'This is an english text.'
#doc = nlp(text)
#print(doc._.language)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def doc_lang(text):
    doc = nlp(text)
    return doc._.language['language']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['PRED_LANG'] = reviews['ALL_TEXT'].apply(doc_lang)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                               REVIEW_HEADLINE  \
0   310163689  BEAUTIFUL BAG                                                  
1   310053706  I love this bag -                                              
2   309950435  It was not broken                                              
3   310088124  I'll never use another company for my water dispenser needs    
4   309938472  Jerky Review                                                   
5   309635349  Beef Jerky Sticks Review                                       
6   310185959  Jerky Review                                                   
7   310336143  Best tea I have had....amazing                                 
8   309765207  Great support on my queries about compatible products.         
9   310150572  the good shizzles                                              
10  320860463  Good so far                                                    
11  310235362  Jerky Review                         

In [11]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0


reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

816

In [14]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

78

In [15]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

125

In [16]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

98981

In [24]:
FP.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID                               REVIEW_HEADLINE  \
1465   315360237  jim beanie                                     
2007   315588207  ALWAYS A GREAT EXPERIENCE AND QUICK  & EASY!   
2111   271714991  STRAWBERRY FEED EVERY SPRING                   
3207   271393150  mNGkZEZH                                       
3208   271202697  fJskQGcl                                       
3209   271195879  vZInPAcU                                       
3210   271413513  PYETXWAk                                       
3212   271483522  OdphymAA                                       
3879   264288603  Tevreden!                                      
4101   264319705  Excellent                                      
4288   264321410  方便使用價格親民                                       
4289   264320940  還不錯，價錢貴了點                                      
4290   264298653  中庸                                             
4291   264324082  放化妝品很好用，可惜價格有一點偏貴                              
4292   264

In [19]:
detect('good good stuff')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [28]:
FP.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 638 to 8681798
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UGC_ID               6456 non-null   int64 
 1   NAME                 6456 non-null   object
 2   REVIEW_HEADLINE      6456 non-null   object
 3   REVIEW_COMMENTS      6456 non-null   object
 4   CR                   6456 non-null   int64 
 5   ALL_TEXT             6456 non-null   object
 6   NAME_1ST_PART_RAW    6456 non-null   object
 7   NAME_1ST_PART        6456 non-null   object
 8   ALL_TEXT_2           6456 non-null   object
 9   ALL_TEXT_3           6456 non-null   object
 10  ALL_TEXT_4           6456 non-null   object
 11  ALL_TEXT_5           6456 non-null   object
 12  ALL_TEXT_6           6456 non-null   object
 13  ALL_TEXT_7           6456 non-null   object
 14  regex_retailer       6456 non-null   object
 15  retailer_not_client  6456 non-null   bool  
 16  m

In [29]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART_RAW", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("ALL_TEXT_2", StringType(), True)\
,StructField("ALL_TEXT_3", StringType(), True)\
,StructField("ALL_TEXT_4", StringType(), True)\
,StructField("ALL_TEXT_5", StringType(), True)\
,StructField("ALL_TEXT_6", StringType(), True)\
,StructField("ALL_TEXT_7", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FP_test4").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…